In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import torch
from torch.optim import SGD

## Access src directory from ./notebooks/ folder
from pathlib import Path
sys.path.append(str(Path('.').absolute().parent))

from src.Snip import SNIP
from src.tools import n_layer_nn

### Manual test with MNIST

In [ ]:
#sgd_val_loss, sgd_accuracy = n_layer_nn(SGD, learning_rate=0.1)

params = torch.Tensor((3,3))
snip = SNIP(params)
#C = snip.prune()
# to multiply weights by C : https://discuss.pytorch.org/t/how-can-i-modify-certain-layers-weight-and-bias/11638